# [****  목차   ****]

## < 문제 명세 >

- 문제 명세


## < Settings >

- import 및 경로 지정
- 피클된 데이터 꺼내오기
- dirty .txt file preprocessed to pretty .csv file
- crosstab
- data split[training, validation, test]


## < analysis >


#### [1] input

- question by question correlation
: pearson, kendall Tau, 단순유사도
- User by User correlation
: pearson, kendall Tau, 단순유사도


#### [2] Score Function

- collaborative filtering with neural network


#### [3] Loss Function

- cross entropy with softmax

*********

*********

*********

*********

# 문제 명세

#### 심리테스트 고객 오ㅇㅇ님이 question 37에 문제에 Yes라고 대답할지 No라고 대답할지 예측해보기?

- 어떤 상황이든지 예측을 하려면, 오 ㅇㅇ 고객으로부터 데이터를 받아야 합니다. 마치 영화추천 사이트가 새로운 가입한 고객으로부터 그 고객이 좋아하는 영화에 대한 도메인 지식을 받는 것처럼요.



- 하지만, 영화추천 사이트에서 만들어야하는 밸류와 제가 만들어내야 하는 밸류는 다릅니다. 주어진 데이터가 다르기 때문입니다. 영화추천 사이트는 이빠진 데이터에를 채우는 것이 밸류의 원천입니다. 하지만, 제게 주어진 데이터는 이빠진 데이터가 아닙니다. 49192명이 100문제씩 완벽하게 풀어낸 Full 데이터셋입니다. 제가 가지고 있는 데이터셋으로부터는 다른 밸류를 창출해야 했습니다.



- 그래서 제가 생각했던 것은 고객으로부터 데이터 입력을 최소화하는 것이었습니다. 고객으로부터 받아내는 데이터의 양을 최소화해보는 것이 해결해보고 싶은 문제였습니다. 필요한 최소한의 질문만 하고서, 고객으로부터 최대한의 정보를 끌어내는 것이 목표였습니다. 다시 말해서, 100문제를 다 물어보지 않고, 고객의 성향을 유추할 수 있는 핵심 문제(10~30여개)만 물어보고서, 고객이 다른 질문에 대해서 어떻게 답변할지 예측해내는 것이 목표였습니다.



- 어떻게 그 핵심 문제 10 ~ 30 개 를 골라낼 것인가. 이것이 해결하고 싶은 문제였습니다. 해당 10 ~ 30 문제는 나머지 70 ~ 90 문제를 예측할 수 있는 질문들이어야 합니다.



- 문제 별로 양의 상관 관계, 음의 상관 관계 두 가지를 바탕으로 예측할 수 있을 것입니다. 양의 상관관계는 예를 들어, "Q3. 영양제를 챙겨 드십니까?"에 yes라고 대답한 사람은 "Q54. 건강을 챙기는 것이 중요합니까?"라는 질문에도 yes라고 대답할 확률이 높은 것처럼 말입니다. 음의 상관관계는 반대의 경우가 되겠지요. 예를 들어, 앞선 Q3에 yes라고 한 사람은 "Q75. 건강을 챙기는 것이 불필요합니까?"라는 질문에 No라고 할 것입니다.



- 또한, 유저들을 클러스터링 해서 예측할 수 있을 것입니다. 예를 들어 "Q3. 아침 식사를 하십니까?"에 yes라고 대답한 A가 ㄲㄲㄲ유형에 속한다면, ㄲㄲㄲ유형에 속하는 또 다른 유저 B도 해당 질문에 yes라고 대답할 확률이 높아질 것입니다.

### 1줄 요약: 100문제가 아닌 10~20여 문제만 물어봐도 나머지 문제에 대한 추측이 가능하도록!!!!! 핵심 문제를 추려내는 것이 이 데이터셋으로부터 얻을 수 있는 가치다.

***

***

***

***

# settings

#### [1/5] import 및 경로 지정

In [1]:
from __future__ import division, print_function

from theano.sandbox import cuda

import theano
from theano import shared, tensor as T
from theano.tensor.nnet import conv2d, nnet
from theano.tensor.signal import pool

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Merge
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam

from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from keras import regularizers

from sklearn.decomposition import PCA

import pandas as pd
import numpy as np
import scipy as cp
import sklearn as sk
from scipy import stats
from scipy.sparse import csr_matrix
import seaborn as sns

path = "/Users/choelinbeom/courses/deeplearning1/data/mind_user_answers/workfile.csv"
model_path = "/Users/choelinbeom/courses/deeplearning1/data/mind_user_answers/model/"

Using TensorFlow backend.


In [2]:
##ratings = pd.read_csv(path)

#### [2/5] pickle된 데이터 꺼내오기

In [3]:
ratings = pd.read_pickle(model_path + 'ratings.pkl')

array_pd = pd.read_pickle(model_path + 'array_pd.pkl')
cross = pd.read_pickle(model_path + 'cross.pkl')

QbyQ_Yes = pd.read_pickle(model_path + 'Questions_by_Questions.pkl')
QbyQ_No = pd.read_pickle(model_path + 'Questions_by_Questions_not.pkl')
QbyQ = QbyQ_Yes + QbyQ_No

In [54]:
QbyQ = QbyQ_Yes + QbyQ_No

In [8]:
##for show
##ratings
##array_pd
##cross
##QbyQ

Questions,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
Questions,,,,,,,,,,,,,,,,,,,,,
1,45192,21497,20168,24144,23720,21580,25513,24404,18864,23527,...,20497,22608,24048,25222,24830,23622,26351,22984,23300,21535
2,21497,45192,27329,17387,21923,26467,17718,20911,23281,22288,...,22452,22721,20369,20497,18505,22437,19486,26695,17007,26352
3,20168,27329,45192,16278,20942,29612,13241,21006,23024,17173,...,21111,26936,18126,17788,14822,24678,15329,29228,22486,30087
4,24144,17387,16278,45192,23090,16310,30893,24808,23562,27727,...,24787,19150,25510,28990,31174,21728,29781,15922,24080,16221
5,23720,21923,20942,23090,45192,22366,24555,24108,22790,24207,...,23605,22028,25120,23018,24006,23108,23997,22084,22280,21181
6,21580,26467,29612,16310,22366,45192,14825,22294,20664,13603,...,18883,30978,18752,17876,15364,28548,14221,34142,24886,32497
7,25513,17718,13241,30893,24555,14825,45192,24877,23723,30876,...,25562,16737,28301,29099,32965,20305,31600,14873,22295,13502
8,24404,20911,21006,24808,24108,22294,24877,45192,22086,22599,...,22343,25084,23952,25202,25450,24624,23927,22396,24808,22297
9,18864,23281,23024,23562,22790,20664,23723,22086,45192,27179,...,27065,19410,24076,23112,24430,20528,23313,19750,20558,20683


#### [3/5] dirty .txt file preprocessed to pretty .csv file

In [ ]:
ratings = ratings.drop('2015-11-16.1', axis=1)
ratings = ratings.drop('20:50:07', axis=1)
ratings = ratings.rename(index=str, columns={"U9": "Users_ID", "2015-11-16": "Questions", "20:44:38" : "Y_or_N"})
ratings = ratings[(ratings.Y_or_N == 'Y') | (ratings.Y_or_N == 'N')]

ratings['Users_ID'] = ratings['Users_ID'].str.replace(r'\D+', '').astype('int')
ratings['Questions'] = ratings['Questions'].str.replace(r'\D+', '').astype('int')
ratings['Y_or_N'] = ratings['Y_or_N'].str.replace('N', '0')
ratings['Y_or_N'] = ratings['Y_or_N'].str.replace('Y', '1')
ratings['Y_or_N'] = ratings['Y_or_N'].str.replace(r'\D+', '').astype('int')

ratings = ratings.sort(['Questions','Users_ID'], ascending=[True, True])

ratings.to_pickle(model_path + 'ratings.pkl')
ratings = pd.read_pickle(model_path + 'ratings.pkl')

#### [4/5] Crosstab

In [14]:
cross = pd.crosstab(ratings.Users_ID, ratings.Questions, ratings.Y_or_N , aggfunc=np.sum)
cross.to_pickle(model_path + 'cross.pkl')
cross = pd.read_pickle(model_path + 'cross.pkl')

cross_array = cross.as_matrix()
cross_array.as.savetxt(model_path + 'cross_array.pkl')

users = ratings.Users_ID.unique()
questions = ratings.Questions.unique()

array_pd = pd.DataFrame(cross_array, index=users, columns=questions)
array_pd.to_pickle(model_path + 'array_pd.pkl')
array_pd = pd.read_pickle(model_path + 'array_pd.pkl')

#### [5/5] QbyQ and UbyU

Questions_by_Questions = cross.transpose().dot(cross)
Questions_by_Questions_not = cross_not.transpose().dot(cross_not)
Questions_by_Questions_not.to_pickle(model_path +'Questions_by_Questions_not.pkl')
Questions_by_Questions.to_pickle(model_path + 'Questions_by_Questions.pkl')

user_by_user_not = cross_not.dot(cross_not.transpose())
unit1 = user_by_user.loc[:4000]
unit2 = user_by_user.loc[4001:8000]
unit3 = user_by_user.loc[8001:12000]
unit4 = user_by_user.loc[12001:16000]
unit5 = user_by_user.loc[16001:20000]
unit6 = user_by_user.loc[20001:24000]
unit7 = user_by_user.loc[24001:28000]
unit8 = user_by_user.loc[28001:32000]
unit9 = user_by_user.loc[32001:36000]
unit10 = user_by_user.loc[36001:40000]
unit11 = user_by_user.loc[40001:44000]
unit12 = user_by_user.loc[44001:48000]
unit13 = user_by_user.loc[48001:52000]
unit14 = user_by_user.loc[52001:]
unit1.to_pickle(model_path + 'user_by_user_u1.pkl')
unit2.to_pickle(model_path + 'user_by_user_u2.pkl')
unit3.to_pickle(model_path + 'user_by_user_u3.pkl')
unit4.to_pickle(model_path + 'user_by_user_u4.pkl')
unit5.to_pickle(model_path + 'user_by_user_u5.pkl')
unit6.to_pickle(model_path + 'user_by_user_u6.pkl')
unit7.to_pickle(model_path + 'user_by_user_u7.pkl')
unit8.to_pickle(model_path + 'user_by_user_u8.pkl')
unit9.to_pickle(model_path + 'user_by_user_u9.pkl')
unit10.to_pickle(model_path + 'user_by_user_u10.pkl')
unit11.to_pickle(model_path + 'user_by_user_u11.pkl')
unit12.to_pickle(model_path + 'user_by_user_u12.pkl')
unit13.to_pickle(model_path + 'user_by_user_u13.pkl')
unit14.to_pickle(model_path + 'user_by_user_u14.pkl')

user_by_user_not = cross_not.dot(cross_not.transpose())
not_unit1 = user_by_user_not.loc[:4000]
not_unit2 = user_by_user_not.loc[4001:8000]
not_unit3 = user_by_user_not.loc[8001:12000]
not_unit4 = user_by_user_not.loc[12001:16000]
not_unit5 = user_by_user_not.loc[16001:20000]
not_unit6 = user_by_user_not.loc[20001:24000]
not_unit7 = user_by_user_not.loc[24001:28000]
not_unit8 = user_by_user_not.loc[28001:32000]
not_unit9 = user_by_user_not.loc[32001:36000]
not_unit10 = user_by_user_not.loc[36001:40000]
not_unit11 = user_by_user_not.loc[40001:44000]
not_unit12 = user_by_user_not.loc[44001:48000]
not_unit13 = user_by_user_not.loc[48001:52000]
not_unit14 = user_by_user_not.loc[52001:]

not_unit1.to_pickle(model_path + 'user_by_user_not_u1.pkl')
not_unit2.to_pickle(model_path + 'user_by_user_not_u2.pkl')
not_unit3.to_pickle(model_path + 'user_by_user_not_u3.pkl')
not_unit4.to_pickle(model_path + 'user_by_user_not_u4.pkl')
not_unit5.to_pickle(model_path + 'user_by_user_not_u5.pkl')
not_unit6.to_pickle(model_path + 'user_by_user_not_u6.pkl')
not_unit7.to_pickle(model_path + 'user_by_user_not_u7.pkl')
not_unit8.to_pickle(model_path + 'user_by_user_not_u8.pkl')
not_unit9.to_pickle(model_path + 'user_by_user_not_u9.pkl')
not_unit10.to_pickle(model_path + 'user_by_user_not_u10.pkl')
not_unit11.to_pickle(model_path + 'user_by_user_not_u11.pkl')
not_unit12.to_pickle(model_path + 'user_by_user_not_u12.pkl')
not_unit13.to_pickle(model_path + 'user_by_user_not_u13.pkl')
not_unit14.to_pickle(model_path + 'user_by_user_not_u14.pkl')

# Dataset Split
training set, validation set, test set을 대략 60, 20, 20으로 구분했습니다.

In [4]:
msk = np.random.rand(len(ratings)) < 0.9
trn_and_test = ratings[msk]
val = ratings[~msk]

msk2 = np.random.rand(len(trn_and_test)) < 0.9
trn = trn_and_test[msk2]
test = trn_and_test[~msk2]

In [79]:
# sort the dataframe
ratings.sort(columns=['Users_ID'], inplace=True)
# set the index to be this and don't drop
ratings.set_index(keys=['Users_ID'], drop=False,inplace=True)
# get a list of names
names=ratings['Users_ID'].unique().tolist()

# now we can perform a lookup on a 'view' of the dataframe
# now you can query all 'joes'
trn = ratings.loc[(ratings.Users_ID>=1) & (ratings.Users_ID<=32000)]
val = ratings.loc[(ratings.Users_ID>32000) & (ratings.Users_ID<=44000)]
test = ratings.loc[(ratings.Users_ID>44000)]
len(test) + len(trn) + len(val)

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


4519200

In [ ]:
a = trn.sort('Users_ID', ascending= True)
a

categorical하게 풀 수 있도록 yes no값을 [0,1] [1,0] 으로 바꾸었습니다.

In [5]:
train_category = to_categorical(trn.Y_or_N)
val_category = to_categorical(val.Y_or_N)
test_category = to_categorical(test.Y_or_N)

In [14]:
trn_tab = pd.crosstab(trn.Users_ID, trn.Questions, trn.Y_or_N , aggfunc=np.sum)
trn_tab

Questions,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
Users_ID,,,,,,,,,,,,,,,,,,,,,
9,1.0,0.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,...,1.0,NaN,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
13,NaN,NaN,1.0,0.0,1.0,NaN,0.0,0.0,1.0,0.0,...,0.0,NaN,1.0,0.0,0.0,NaN,0.0,1.0,1.0,0.0
14,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
15,NaN,0.0,NaN,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,NaN,NaN,0.0,1.0,0.0,NaN,1.0,1.0,1.0,0.0
16,NaN,0.0,1.0,1.0,0.0,NaN,0.0,1.0,0.0,NaN,...,0.0,1.0,1.0,NaN,1.0,1.0,NaN,0.0,1.0,1.0
17,1.0,NaN,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN,...,1.0,1.0,1.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0
20,0.0,1.0,1.0,0.0,0.0,NaN,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,NaN,0.0,NaN,0.0
21,1.0,1.0,NaN,0.0,NaN,1.0,0.0,1.0,1.0,1.0,...,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
22,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.0,NaN,0.0,1.0,0.0,0.0,NaN,0.0,0.0


# Correlations

#### (1) pearson corrleation

In [11]:
correlation_matrix = array_pd.corr(method='pearson')
pearson_cor_10_largest = correlation_matrix[1].nlargest(10)

for i in range(99) : 
    pearson_cor_10_largest = pearson_cor_10_largest.append(correlation_matrix[i+2].nlargest(10))

sliced_pearson_cor = []

for i in range(0, len(pearson_cor_10_largest.as_matrix()), 10):
    sliced_pearson_cor.append(pearson_cor_10_largest.as_matrix()[i : i+10])

np_sliced_pearson_cor = np.array(sliced_pearson_cor)

In [ ]:
np_sliced_pearson_cor

#### (2) kendall's Tau corrleation

In [117]:
kendall_correlation_matrix = array_pd.corr(method='kendall')

kendall_cor_10_largest = kendall_correlation_matrix[1].nlargest(10)

for i in range(99) : 
    kendall_cor_10_largest = kendall_cor_10_largest.append(kendall_correlation_matrix[i+2].nlargest(10))

sliced_kendall_cor = []

for i in range(0, len(kendall_cor_10_largest.as_matrix()), 10):
    sliced_pearson_cor.append(kendall_cor_10_largest.as_matrix()[i : i+10])

np_sliced_kendall_cor = np.array(sliced_kendall_cor)

In [ ]:
np_sliced_kendall_cor

#### (3) neighbor based correlation

In [56]:
UbyU1 = pd.read_pickle(model_path + 'user_by_user_u1.pkl')
UbyUnot1 = pd.read_pickle(model_path + 'user_by_user_not_u1.pkl')

In [66]:
UbyU = UbyU1+UbyUnot1

In [ ]:
QbyQ -= np.mean(QbyQ, axis = 1)
QbyQ /= np.std(QbyQ, axis = 1)

UbyU -= np.mean(UbyU, axis = 1)
UbyU /= np.std(UbyU, axis = 1)

In [67]:
QbyQ.shape

(100, 100)

In [68]:
UbyU.shape

(3313, 45192)

#### (4) cosign similarity

# Data Analysis

#### (1) Classical Dot Collaborative Filtering

In [55]:
n_users = trn.shape[0]
n_questions = 100
n_factors = 50

user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(user_in)

questions_in = Input(shape=(1,), dtype='int64', name='questions_in')
m = Embedding(n_questions+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(questions_in)

In [ ]:
x1 = keras.layers.merge([u, m], mode='dot')
x1 = Flatten()(x1)
x1 = Dense(2)(x1)

x1 = Activation('softmax')(x1)
model = Model([user_in, questions_in], x1)
model.compile(Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, epochs=2, 
          validation_data=([val.Users_ID, val.Questions], val_category))

In [20]:
model.optimizer.lr=0.01

In [21]:
model.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, epochs=1, 
          validation_data=([val.Users_ID, val.Questions], val_category))

Train on 2640449 samples, validate on 903821 samples
Epoch 1/1
2640449/2640449 [==============================] - 261s - loss: 1.4457 - acc: 0.5584 - val_loss: 0.9860 - val_acc: 0.5586


In [22]:
model.optimizer.lr=0.001

In [23]:
model.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, epochs=4, 
          validation_data=([val.Users_ID, val.Questions], val_category))

Train on 2640449 samples, validate on 903821 samples
Epoch 1/4
2640449/2640449 [==============================] - 207s - loss: 0.8442 - acc: 0.5584 - val_loss: 0.7363 - val_acc: 0.5586
Epoch 2/4
2640449/2640449 [==============================] - 190s - loss: 0.7112 - acc: 0.5584 - val_loss: 0.6944 - val_acc: 0.5586
Epoch 3/4
2640449/2640449 [==============================] - 189s - loss: 0.6911 - acc: 0.5584 - val_loss: 0.6888 - val_acc: 0.5586
Epoch 4/4
2640449/2640449 [==============================] - 207s - loss: 0.6882 - acc: 0.5584 - val_loss: 0.6876 - val_acc: 0.5586


In [24]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
questions_in (InputLayer)        (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 1, 50)         132022500   user_in[0][0]                    
____________________________________________________________________________________________________
embedding_4 (Embedding)          (None, 1, 50)         5050        questions_in[0][0]               
___________________________________________________________________________________________

In [50]:
after_softmax = model.predict([test.Users_ID,test.Questions], batch_size = 200000)

In [73]:
after_softmax

array([[ 0.42808649,  0.57191354],
       [ 0.42838645,  0.57161355],
       [ 0.42795101,  0.57204902],
       ..., 
       [ 0.42777911,  0.57222092],
       [ 0.42817867,  0.57182139],
       [ 0.42859548,  0.57140452]], dtype=float32)

In [41]:
to_categorical(after_softmax.argmax(axis=1))

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [ ]:
model.evaluate([test.Users_ID,test.Questions], test.Y_or_N, batch_size = 200000)

#### (2) classical neural network

In [15]:
n_users = trn.shape[0]
n_questions = 100
n_factors = 50

user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(user_in)

questions_in = Input(shape=(1,), dtype='int64', name='questions_in')
m = Embedding(n_questions+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(questions_in)

In [17]:
x2 = merge([u, m], mode='concat')
x2 = Flatten()(x2)
x2 = Dropout(0.3)(x2)
x2 = Dense(70, activation='relu')(x2)
x2 = Dropout(0.75)(x2)
x2 = Dense(1)(x2)
x2 = Dropout(0.3)(x2)
x2 = Dense(2)(x2)
x2 = Activation('softmax')(x2)

model_nn = Model([user_in, questions_in], x2)
model_nn.compile(SGD(0.001), loss='binary_crossentropy', metrics=['accuracy'])

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if __name__ == '__main__':


In [18]:
model_nn.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, nb_epoch=8, 
          validation_data=([val.Users_ID, val.Questions], val_category))

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app
/Users/choelinbeom/anaconda/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:88: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 183089900 elements. This may consume a large amount of memory.
  "This may consume a large amount of memory." % num_elements)


Train on 3661797 samples, validate on 451258 samples
Epoch 1/8
3661797/3661797 [==============================] - 226s - loss: 15.7725 - acc: 0.5178 - val_loss: 15.7723 - val_acc: 0.5142
Epoch 2/8
3661797/3661797 [==============================] - 242s - loss: 15.7723 - acc: 0.5215 - val_loss: 15.7721 - val_acc: 0.5293
Epoch 3/8
3661797/3661797 [==============================] - 300s - loss: 15.7720 - acc: 0.5248 - val_loss: 15.7719 - val_acc: 0.5419
Epoch 4/8
3661797/3661797 [==============================] - 294s - loss: 15.7718 - acc: 0.5277 - val_loss: 15.7716 - val_acc: 0.5505
Epoch 5/8
3661797/3661797 [==============================] - 282s - loss: 15.7715 - acc: 0.5310 - val_loss: 15.7714 - val_acc: 0.5556
Epoch 6/8
3661797/3661797 [==============================] - 261s - loss: 15.7714 - acc: 0.5336 - val_loss: 15.7712 - val_acc: 0.5581
Epoch 7/8
3661797/3661797 [==============================] - 297s - loss: 15.7711 - acc: 0.5363 - val_loss: 15.7709 - val_acc: 0.5591
Epoch 8/8

In [86]:
model_nn.optimizer.lr=0.01

In [ ]:
model_nn.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, nb_epoch=2, 
          validation_data=([val.Users_ID, val.Questions], val_category))

In [19]:
softmax_result = model_nn.predict([test.Users_ID,test.Questions], batch_size =200000)
softmax_result

array([[ 0.49079546,  0.50920451],
       [ 0.49282321,  0.50717676],
       [ 0.493507  ,  0.50649297],
       ..., 
       [ 0.49167562,  0.50832438],
       [ 0.49246827,  0.5075317 ],
       [ 0.49208143,  0.50791854]], dtype=float32)

In [20]:
result = np.around(softmax_result)

In [ ]:
nom = np.sum(np.dot(result,test_category.T))

In [ ]:
denom = len(test_category)

In [ ]:
final_acc = float(nom)/denom
final_acc

In [ ]:
model.summary()

(2-1) classical neural network - with adam

In [6]:
n_users = trn.shape[0]
n_questions = 100
n_factors = 50

user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(user_in)

questions_in = Input(shape=(1,), dtype='int64', name='questions_in')
m = Embedding(n_questions+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(questions_in)

In [8]:
x2_1 = merge([u, m], mode='concat')
x2_1 = Flatten()(x2_1)
x2_1 = Dropout(0.3)(x2_1)
x2_1 = Dense(70, activation='relu')(x2_1)
x2_1 = Dropout(0.75)(x2_1)
x2_1 = Dense(1)(x2_1)
x2_1 = Dropout(0.3)(x2_1)
x2_1 = Dense(2)(x2_1)
x2_1 = Activation('softmax')(x2_1)

model_nn = Model([user_in, questions_in], x2_1)
model_nn.compile(Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if __name__ == '__main__':


In [9]:
model_nn.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, nb_epoch=4, 
          validation_data=([val.Users_ID, val.Questions], val_category))

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app
/Users/choelinbeom/anaconda/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:88: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 182996650 elements. This may consume a large amount of memory.
  "This may consume a large amount of memory." % num_elements)


Train on 3659932 samples, validate on 451924 samples
Epoch 1/4
3659932/3659932 [==============================] - 695s - loss: 9.8242 - acc: 0.5580 - val_loss: 4.6808 - val_acc: 0.5670
Epoch 2/4
3659932/3659932 [==============================] - 648s - loss: 2.7267 - acc: 0.6002 - val_loss: 1.2960 - val_acc: 0.6462
Epoch 3/4
3659932/3659932 [==============================] - 639s - loss: 0.9336 - acc: 0.6178 - val_loss: 0.6926 - val_acc: 0.6454
Epoch 4/4
3659932/3659932 [==============================] - 602s - loss: 0.6723 - acc: 0.6199 - val_loss: 0.6367 - val_acc: 0.6489


In [10]:
model_nn.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, nb_epoch=4, 
          validation_data=([val.Users_ID, val.Questions], val_category))

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 3659932 samples, validate on 451924 samples
Epoch 1/4
3659932/3659932 [==============================] - 625s - loss: 0.6501 - acc: 0.6245 - val_loss: 0.6304 - val_acc: 0.6562
Epoch 2/4
3659932/3659932 [==============================] - 586s - loss: 0.6457 - acc: 0.6312 - val_loss: 0.6274 - val_acc: 0.6620
Epoch 3/4
3659932/3659932 [==============================] - 533s - loss: 0.6436 - acc: 0.6341 - val_loss: 0.6252 - val_acc: 0.6650
Epoch 4/4
3659932/3659932 [==============================] - 573s - loss: 0.6420 - acc: 0.6355 - val_loss: 0.6231 - val_acc: 0.6655


In [23]:
model_nn.optimizer.lr=0.0001

In [ ]:
model_nn.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, nb_epoch=2, 
          validation_data=([val.Users_ID, val.Questions], val_category))

In [11]:
softmax_result = model_nn.predict([test.Users_ID,test.Questions], batch_size =200000)
softmax_result

array([[ 0.48165077,  0.51834929],
       [ 0.5470255 ,  0.45297453],
       [ 0.53223163,  0.4677684 ],
       ..., 
       [ 0.7566371 ,  0.24336289],
       [ 0.74711603,  0.25288397],
       [ 0.77893507,  0.22106494]], dtype=float32)

In [12]:
result = np.around(softmax_result)
result

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]], dtype=float32)

In [18]:
len(result)

407344

In [19]:
len(test_category)

407344

In [20]:
score = 0

for i in range(len(result)):
    if result[i][0] == test_category[i][0]:
        score+=1

In [21]:
score

271289

In [22]:
float(score) / len(test_category)

0.6659948348324758

#### (3)  neural network 2 [dropout -> batchnormalization || + expanded weights] : Overfitting

In [92]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=regularizers.l2(reg))(inp)

In [94]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
questions_in, m = embedding_input('movie_in', n_questions, n_factors, 1e-4)

In [95]:
def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [97]:
ub = create_bias(user_in, n_users)
qb = create_bias(questions_in, n_questions)

In [6]:
n_users = trn.shape[0]
n_questions = 100
n_factors = 50

user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(user_in)

questions_in = Input(shape=(1,), dtype='int64', name='questions_in')
m = Embedding(n_questions+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(questions_in)

In [7]:
x2 = merge([u, m], mode='concat')
x2 = Flatten()(x2)
x2 = BatchNormalization()(x2)
x2 = Dense(1000, activation='relu')(x2)
x2 = BatchNormalization()(x2)
x2 = Dense(200, activation='relu')(x2)
x2 = BatchNormalization()(x2)
x2 = Dense(200, activation='relu')(x2)
x2 = BatchNormalization()(x2)
x2 = Dense(2)(x2)
x2 = Activation('softmax')(x2)

model_nn2 = Model([user_in, questions_in], x2)
model_nn2.compile(Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if __name__ == '__main__':
/Users/choelinbeom/anaconda/lib/python2.7/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [8]:
model_nn2.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, nb_epoch=8, 
          validation_data=([val.Users_ID, val.Questions], val_category))

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app
/Users/choelinbeom/anaconda/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:88: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 132014850 elements. This may consume a large amount of memory.
  "This may consume a large amount of memory." % num_elements)


Train on 2640296 samples, validate on 903561 samples
Epoch 1/8
2640296/2640296 [==============================] - 1310s - loss: 8.4550 - acc: 0.5923 - val_loss: 5.0750 - val_acc: 0.5588
Epoch 2/8
2640296/2640296 [==============================] - 1424s - loss: 3.4796 - acc: 0.6554 - val_loss: 2.1155 - val_acc: 0.5588
Epoch 3/8
2640296/2640296 [==============================] - 1399s - loss: 1.4721 - acc: 0.6861 - val_loss: 1.1066 - val_acc: 0.5588
Epoch 4/8
1200000/2640296 [============>.................] - ETA: 687s - loss: 0.9054 - acc: 0.7137

KeyboardInterrupt: 

In [ ]:
model_nn.optimizer.lr=0.01

In [ ]:
model_nn2.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, nb_epoch=3, 
          validation_data=([val.Users_ID, val.Questions], val_category))

In [ ]:
model_nn.optimizer.lr=0.001

In [ ]:
model_nn2.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, nb_epoch=8, 
          validation_data=([val.Users_ID, val.Questions], val_category))

#### (4)  neural network 3 [dropout -> batch normalization] 

In [9]:
n_users = trn.shape[0]
n_questions = 100
n_factors = 50

user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(user_in)

questions_in = Input(shape=(1,), dtype='int64', name='questions_in')
m = Embedding(n_questions+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(questions_in)

In [11]:
x4 = merge([u, m], mode='concat')
x4 = Flatten()(x4)
x4 = BatchNormalization()(x4)
x4 = Dense(70, activation='relu')(x4)
x4 = BatchNormalization()(x4)
x4 = Dense(30, activation='relu')(x4)
x4 = BatchNormalization()(x4)
x4 = Dense(10, activation='relu')(x4)
x4 = BatchNormalization()(x4)
x4 = Dense(2)(x4)
x4 = Activation('softmax')(x4)

model_nn3 = Model([user_in, questions_in], x4)
model_nn3.compile(Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if __name__ == '__main__':


In [13]:
model_nn3.fit([trn.Users_ID, trn.Questions], train_category, batch_size=200000, nb_epoch=8, 
          validation_data=([val.Users_ID, val.Questions], val_category))

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 2640296 samples, validate on 903561 samples
Epoch 1/8
2640296/2640296 [==============================] - 301s - loss: 8.5076 - acc: 0.5673 - val_loss: 5.0792 - val_acc: 0.5591
Epoch 2/8
2640296/2640296 [==============================] - 284s - loss: 3.4908 - acc: 0.6493 - val_loss: 2.1152 - val_acc: 0.5588
Epoch 3/8
2640296/2640296 [==============================] - 257s - loss: 1.4850 - acc: 0.6780 - val_loss: 1.1054 - val_acc: 0.5588
Epoch 4/8
 200000/2640296 [=>............................] - ETA: 213s - loss: 1.0046 - acc: 0.6946

KeyboardInterrupt: 

#### (4)  neural network 4 [Matrix Factorization] 

In [97]:
# sort the dataframe
ratings.sort(columns=['Users_ID'], inplace=True)
# set the index to be this and don't drop
ratings.set_index(keys=['Users_ID'], drop=False,inplace=True)
# get a list of names
names=ratings['Users_ID'].unique().tolist()

# now we can perform a lookup on a 'view' of the dataframe
# now you can query all 'joes'
trn = ratings.loc[(ratings.Users_ID>=1) & (ratings.Users_ID<=32000)]
val = ratings.loc[(ratings.Users_ID>32000) & (ratings.Users_ID<=44000)]
test = ratings.loc[(ratings.Users_ID>44000)]
len(test) + len(trn) + len(val)

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


4519200

In [98]:
from scipy.sparse.linalg import svds

trn_cross = pd.crosstab(trn.Users_ID, trn.Questions, trn.Y_or_N , aggfunc=np.sum)
trn_cross.to_pickle(model_path + 'trn_cross.pkl')
trn_cross = pd.read_pickle(model_path + 'trn_cross.pkl')
trn_cross_array = trn_cross.as_matrix()

In [99]:
trn_cross_array

array([[1, 0, 0, ..., 0, 1, 0],
       [0, 1, 1, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [100]:
user_ratings_mean.reshape(-1,1)

array([[ 0.59],
       [ 0.45],
       [ 0.  ],
       ..., 
       [ 0.43],
       [ 0.68],
       [ 0.55]])

In [101]:
user_ratings_mean = np.nanmean(trn_cross_array, axis=1)
trn_cross_array_demeaned = trn_cross_array - user_ratings_mean.reshape(-1,1)
trn_cross_array_demeaned

array([[ 0.41, -0.59, -0.59, ..., -0.59,  0.41, -0.59],
       [-0.45,  0.55,  0.55, ...,  0.55,  0.55, -0.45],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       ..., 
       [-0.43, -0.43,  0.57, ..., -0.43, -0.43, -0.43],
       [ 0.32, -0.68, -0.68, ..., -0.68, -0.68, -0.68],
       [-0.55,  0.45, -0.55, ..., -0.55, -0.55, -0.55]])

In [102]:
U, sigma, Vt =svds(trn_cross_array_demeaned,k=50)
sigma = np.diag(sigma)

all_user_predicted_ratings = np.dot(np.dot(U,sigma), Vt) + user_ratings_mean.reshape(-1,1)
trn_preds_df = pd.DataFrame(all_user_predicted_ratings, columns= trn_cross.columns, index = trn_cross.index)

In [105]:
n_users = trn.shape[0]
n_questions = 100
n_factors = 50

user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(user_in)

questions_in = Input(shape=(1,), dtype='int64', name='questions_in')
m = Embedding(n_questions+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(questions_in)

svds_in = Input(shape=(1,), dtype='int64', name='svds_in')
s = Embedding(n_questions+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(svds_in)

In [106]:
s4 = Flatten()(s)
s4 = Dense(10)(s4)
s4 = Reshape((10,))

x4 = keras.layers.merge([u, m], mode='dot')
x4 = Flatten()(x4)
x4 = Dense(10)(x4)

x4 = keras.layers.merge([x4, s4], mode ='concat')

x4 = Dense(2)(x4)
x4 = Activation('softmax')(x4)

model = Model([user_in, questions_in], x4)
model.compile(Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])


/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


ValueError: Layer merge_37 was called with an input that isn't a symbolic tensor. Received type: <class 'keras.layers.core.Reshape'>. Full input: [<tf.Tensor 'dense_58/BiasAdd:0' shape=(?, 10) dtype=float32>, <keras.layers.core.Reshape object at 0x15b68a990>]. All inputs to the layer should be tensors.

In [ ]:
model.fit([trn_preds_df.Users_ID, trn_preds_df.Questions], train_category, batch_size=200000, epochs=2, 
          validation_data=([val.Users_ID, val.Questions], val_category))

#### NeuralNet

In [25]:
n_users = trn.shape[0]
n_questions = 100
n_factors = 50

user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(user_in)

questions_in = Input(shape=(1,), dtype='int64', name='questions_in')
m = Embedding(n_questions+1, n_factors, input_length=1, embeddings_regularizer=regularizers.l2(1e-4))(questions_in)

In [26]:
model_left = Sequential()
model_left.add(Embedding(n_users+1, n_factors, input_length=1, embeddings_regularizer= regularizers.l2(1e-4)))
model_left.add(Flatten())
model_left.add(Dense(64))
model_left.add(Activation('relu'))
model_left.add(Dense(82))
model_left.add(Activation('relu'))
model_left.add(Dense(10))

model_right = Sequential()
model_right.add(Embedding(n_questions+1, n_factors, input_length=1, embeddings_regularizer= regularizers.l2(1e-4)))
model_right.add(Flatten())
model_right.add(Dense(24))
model_right.add(Activation('relu'))
model_right.add(Dense(30))
model_right.add(Activation('relu'))
model_right.add(Dense(10))

model2 = Sequential()
model2.add(Merge([model_left, model_right], mode='concat'))
model2.add(Dense(10))
model2.add(Activation('relu'))
model2.add(Dense(20))
model2.add(Activation('relu'))
model2.add(Dense(10))
model2.add(Dense(2)) 
model2.add(Activation('softmax'))

model2.compile(Adam(0.001), loss='binary_crossentropy', metrics = ['accuracy'])

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [27]:
model2.fit([trn.Users_ID, trn.Questions], train_category, batch_size=250000, nb_epoch=6, 
          validation_data=([val.Users_ID, val.Questions], val_category))

/Users/choelinbeom/anaconda/lib/python2.7/site-packages/keras/models.py:851: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3659932 samples, validate on 451924 samples
Epoch 1/6
3659932/3659932 [==============================] - 461s - loss: 10.8248 - acc: 0.5547 - val_loss: 6.1662 - val_acc: 0.5583
Epoch 2/6
 750000/3659932 [=====>........................] - ETA: 323s - loss: 5.7650 - acc: 0.5578

KeyboardInterrupt: 

In [35]:
model.save_weights(model_path+'weight_mind_riiid.h5')
model.load_weights(model_path+'weight_mind_riiid.h5')

In [38]:
scores = model.evaluate([test.Users_ID, test.Questions], test.Y_or_N, batch_size =250000)

975552/975552 [==============================] - 5s     


In [40]:
scores

0.214141383292027

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_2 (Merge)              (None, 20)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                210       
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 20)                220       
_________________________________________________________________
activation_12 (Activation)   (None, 20)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 11        
Total para

#### colaborative filtering - binary

In [11]:
unique_users= ratings['Users_ID'].unique()

In [ ]:
pd.pivot_table(ratings,index=["Users_ID"],
               columns=["Questions"])

#### Neighbor_Based_classic

In [ ]:
cross.astype(bool)

In [18]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

n_users = trn.shape[0]
n_questions = 100

n_users

3616013

In [ ]:
cross_not = ~cross.astype(bool)
cross_not = cross_not.astype(int)
cross_not

In [62]:
not_unit1 = user_by_user_not.loc[:4000]

In [52]:
distance = unit1_not + u1

In [ ]:
u1

In [ ]:
distance.applymap(f)

In [70]:
def f(x):
    if x == 45192: return 0
    else :return x

In [13]:
u1.columns

Int64Index([    9,    13,    14,    15,    16,    17,    20,    21,    22,
               26,
            ...
            54095, 54096, 54097, 54098, 54100, 54101, 54102, 54103, 54104,
            54106],
           dtype='int64', name=u'Users_ID', length=45192)

In [26]:
conc_b1 = pd.concat([conc1, conc2])

In [ ]:
conc_b2 = pd.concat([conc3, conc4, conc5])

In [ ]:
concate_complete = pd.concat([conc_b1, conc_b2])

In [ ]:
for i in range(2):
    
    unit_number = i+13
    
    temp = pd.read_pickle(model_path + 'user_by_user_u'+ str(unit_number) + '.pkl')
    
    user_by_user = pd.concat([user_by_user, temp])

In [ ]:
True_by_True_users = np.dot(csr_matrix(cross_array),csr_matrix(cross_array.T))

In [ ]:
Save_Users_array = model_path + 'TruebyTrue.txt'
np.savetxt(Save_Users_array, True_by_True_users, fmt='%d')

In [ ]:
True_by_True_questions = np.dot(cross_array.T,cross_array)
Save_Question_Neighbors_array = model_path + 'TruebyTrue.txt'
np.savetxt(Save_Question_Neighbors_array, True_by_True_questions, fmt='%d')

In [ ]:
False_by_False = np.dot(~cross_array,~cross_array.T)
also_False_by_False = True_by_True + False_by_False
SaveAt2 = model_path + 'TruebyTrue_Plus_FalsebyFalse.txt'
np.savetxt(SaveAt2, also_False_by_False, fmt='%d')

In [99]:


def xor_dot(x,y):
    for i in range(len(x)):
        out = out.append(np.logical_xor(x[i+1],(y[:,i+1])))

    
kk = [[True, True, False],[False, True, False], [True, True, True],[True, False, False]]
kkk = [[False, False, False,False],[False, True, False,False], [False,False ,True, True]]



kk = np.array(kk)
kkk = np.array(kkk)

~kk
kk



array([[ True,  True, False],
       [False,  True, False],
       [ True,  True,  True],
       [ True, False, False]], dtype=bool)

In [17]:
model.save_weights(model_path+'bias.h5')


In [18]:
model.load_weights(model_path+'bias.h5')

In [ ]:
pca = PCA(n_components=3)
movie_pca = pca.fit(movie_emb.T).components_

In [ ]:
fac0 = movie_pca[0]

In [ ]:
movie_comp = [(f, movie_names[movies[i]]) for f,i in zip(fac0, topMovies)]

In [ ]:
sorted(movie_comp, key=itemgetter(0), reverse=True)[:10]
sorted(movie_comp, key=itemgetter(0))[:10]